In [1]:
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.multioutput import MultiOutputRegressor

### Helper Functions

In [2]:
def loadData(file):
    data = pd.read_csv(file)
    print('Raw shape: ',data.shape)
    data['Date'] = pd.to_datetime(data.Date)
    data['DOLocationID'] = data['DOLocationID'].astype(str)
    print('Days: ',len(set(data.Date)))
    return data

In [3]:
def getTimeSeries(df):
    table = pd.pivot_table(df, values='vehicle_count', index=['Date','Hour'],
                    columns=['DOLocationID'], aggfunc=np.sum, fill_value=0)
    return table

In [4]:
def zscoreNormalizeSpatial(matrix):
    m = matrix.copy()
    for i in range(m.shape[0]):
        m[i, :] = (m[i, :] - m[i, :].mean()) / (m[i, :].std()+1e-10)
        
    return m

In [5]:
def standardize(matrix):
    m = matrix.copy()
    scaler = StandardScaler()
    scaler.fit(m)
    t = scaler.transform(m)
    return scaler, t

In [6]:
def inverse_standardize(matrix, scaler):
    t = matrix.copy()
    return scaler.inverse_transform(t)

In [7]:
def addLag(dataset, maxlag, lagColumns):
    dataset_list = [dataset]

    for l in range(1, maxlag+1):
        df = dataset.shift(l)
        df = df[lagColumns]
        df.columns = [c+'_lag_'+str(l) for c in df.columns]
        dataset_list.append(df)

    dataset = pd.concat(dataset_list, axis=1).dropna()
    return dataset

In [8]:
def get_rmse(matrix1, matrix2):
    sumSquareError = np.mean(np.power(matrix1 - matrix2,2))
    rmse = np.power(sumSquareError,0.5)
    return rmse

In [9]:
def get_weights(rawdata, zontoBorough):
    
    rawdata['Borough'] = rawdata['DOLocationID'].apply(lambda x:zontoBorough[x])
    
    borough_df = rawdata[['vehicle_count','Borough']].groupby(by='Borough').sum().reset_index()

    zone_df = rawdata[['vehicle_count','DOLocationID']].groupby(by='DOLocationID').sum().reset_index()

    zone_df['Borough'] = zone_df['DOLocationID'].apply(lambda x:zontoBorough[x])

    zone_df = pd.merge(borough_df, zone_df, on=['Borough'], how='inner')

    zone_df['zone_weight'] = zone_df.vehicle_count_y / zone_df.vehicle_count_x

    zone_df = zone_df[['Borough', 'DOLocationID', 'zone_weight']]

    return zone_df

#### Load Raw Data

In [10]:
hub = 'Jfk'
tune_hyp_params = True

In [11]:
dataDir = '/home/urwa/Documents/Projects/NYU Remote/project/data/processedData/'
file = dataDir + hub + 'VehiceByHour.csv'

In [12]:
rawdata = loadData(file)

Raw shape:  (2260080, 4)
Days:  365


In [13]:
rawdata.head(2)

,DOLocationID,Date,Hour,vehicle_count
0,1,2018-01-01,0,1.0
1,2,2018-01-01,0,0.0


In [14]:
edge_data = getTimeSeries(rawdata)
edge_data = edge_data.reset_index()
edge_data.head(3)

DOLocationID,Date,Hour,1,10,100,101,102,106,107,108,...,90,91,92,93,94,95,96,97,98,99
0,2018-01-01,0,1,7,0,0,0,1,2,1,...,3,1,1,0,1,6,0,1,0,0
1,2018-01-01,1,0,4,0,1,0,1,4,0,...,3,4,1,0,0,2,0,0,0,0
2,2018-01-01,2,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


### Merge External Data Features

In [15]:
externalDataDir = "/home/urwa/Documents/Projects/NYU Remote/project/data/HongData/"
extFile = externalDataDir + hub.upper() + ".csv"

In [16]:
extDf = pd.read_csv(extFile)
print(extDf.shape)
extDf.head(2)

(8760, 46)


,date,arrival,fhv,yellow,vehicle,ifmon,iftue,ifwed,ifthu,iffri,...,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,18/1/1 0:00,6,263,174,437,1,0,0,0,0,...,18,7,12.5,-21.2,52,0,0.0,0.0,0,0
1,18/1/1 1:00,6,138,133,271,1,0,0,0,0,...,18,7,12.5,-21.2,52,0,0.0,0.0,0,0


In [17]:
extDf['date'] = pd.to_datetime(extDf['date'], yearfirst=True)
extDf.head(2)

,date,arrival,fhv,yellow,vehicle,ifmon,iftue,ifwed,ifthu,iffri,...,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,2018-01-01 00:00:00,6,263,174,437,1,0,0,0,0,...,18,7,12.5,-21.2,52,0,0.0,0.0,0,0
1,2018-01-01 01:00:00,6,138,133,271,1,0,0,0,0,...,18,7,12.5,-21.2,52,0,0.0,0.0,0,0


In [18]:
min(extDf.date), max(extDf.date)

(Timestamp('2018-01-01 00:00:00'), Timestamp('2018-12-31 23:00:00'))

In [19]:
extDf['Hour'] = extDf['date'].dt.hour
extDf['Dow'] = extDf['date'].dt.dayofweek
extDf['Date'] = extDf['date'].dt.date

In [20]:
extDf.columns

Index(['date', 'arrival', 'fhv', 'yellow', 'vehicle', 'ifmon', 'iftue',
       'ifwed', 'ifthu', 'iffri', 'ifsat', 'ifsun', 'if0', 'if1', 'if2', 'if3',
       'if4', 'if5', 'if6', 'if7', 'if8', 'if9', 'if10', 'if11', 'if12',
       'if13', 'if14', 'if15', 'if16', 'if17', 'if18', 'if19', 'if20', 'if21',
       'if22', 'if23', 'maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow', 'Hour', 'Dow',
       'Date'],
      dtype='object')

In [21]:
selected_columns = ['Date', 'Hour', 'Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

In [22]:
extDf = extDf[selected_columns]

In [23]:
print(edge_data.shape)
print(extDf.shape)

(8760, 260)
(8760, 14)


In [24]:
edge_data['Date'] = pd.to_datetime(edge_data['Date'])
extDf['Date'] = pd.to_datetime(extDf['Date'])

In [25]:
edge_data = pd.merge(edge_data,extDf, on=['Date', 'Hour'], how='inner')
print(edge_data.shape)
edge_data['Date'] = edge_data['Date'].dt.date
edge_data.head()

(8760, 272)


,Date,Hour,1,10,100,101,102,106,107,108,...,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,2018-01-01,0,1,7,0,0,0,1,2,1,...,18,7,12.5,-21.2,52,0,0.0,0.0,0,0
1,2018-01-01,1,0,4,0,1,0,1,4,0,...,18,7,12.5,-21.2,52,0,0.0,0.0,0,0
2,2018-01-01,2,0,1,0,0,0,0,0,0,...,18,7,12.5,-21.2,52,0,0.0,0.0,0,0
3,2018-01-01,3,0,0,0,0,0,0,0,0,...,18,7,12.5,-21.2,52,0,0.0,0.0,0,0
4,2018-01-01,4,0,3,0,0,1,0,0,1,...,18,7,12.5,-21.2,52,0,0.0,0.0,0,0


In [26]:
edge_data.columns

Index(['Date', 'Hour', '1', '10', '100', '101', '102', '106', '107', '108',
       ...
       'maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd', 'cdd',
       'participation', 'newsnow', 'snowdepth', 'ifSnow'],
      dtype='object', length=272)

In [28]:
DateColumns = ['Date']

ext_columns = ['Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

targetColumns = [c for c in edge_data.columns if c not in ext_columns and \
                c not in DateColumns and c != 'Hour']

lagColumns = targetColumns + ['arrival']

In [29]:
maxlag = 12

edge_data_lag = addLag(edge_data, maxlag, lagColumns)

edge_data_lag.shape

(8748, 3380)

In [32]:
CommR2List = []
EdgeR2List = []
residualDf_list = []

for m in range(1,13):
    print()
    print("month: ",m)
    month_index  = pd.to_datetime(edge_data_lag.Date).dt.month == m

    dataset_train = edge_data_lag[~month_index]
    dataset_test = edge_data_lag[month_index]
    print("Train Size: ",dataset_train.shape)
    print("Test Size: ",dataset_test.shape)

    X_train = dataset_train.drop(targetColumns+DateColumns , axis = 1)
    X_test = dataset_test.drop(targetColumns+DateColumns , axis = 1)
    y_train = dataset_train[targetColumns]
    y_test = dataset_test[targetColumns]

    rf2 = RandomForestRegressor(random_state = 2019, n_estimators=150, 
                               min_samples_split=3,
                               min_samples_leaf= 2, 
                               max_features= 'sqrt',
                               max_depth= None, 
                               bootstrap= False)

    rf2.fit(X_train,y_train)

    print("Train R2: ",rf2.score(X_train,y_train))
    test_r2 = rf2.score(X_test,y_test)
    print("Test R2: ",test_r2)


    prediction = rf2.predict(X_test)


    residual = y_test - prediction
    residual_df = dataset_test[['Date','Hour']]
    residual_df = pd.concat([residual_df,pd.DataFrame(residual)], axis =1)


    edge_r2 = r2_score(y_test, prediction, multioutput='variance_weighted')
    print("Edge R2: ",edge_r2)

    CommR2List.append(test_r2)
    EdgeR2List.append(edge_r2)
    residualDf_list.append(residual_df)


month:  1
Train Size:  (8016, 3380)
Test Size:  (732, 3380)
Train R2:  0.9065706515987042
Test R2:  0.48325367467909514
Edge R2:  0.48325367467909514

month:  2
Train Size:  (8076, 3380)
Test Size:  (672, 3380)
Train R2:  0.9059554554879414
Test R2:  0.5194713635970322
Edge R2:  0.5194713635970322

month:  3
Train Size:  (8004, 3380)
Test Size:  (744, 3380)
Train R2:  0.9058276503574109
Test R2:  0.5581892472246949
Edge R2:  0.5581892472246949

month:  4
Train Size:  (8028, 3380)
Test Size:  (720, 3380)
Train R2:  0.9057519221460241
Test R2:  0.5713376472760041
Edge R2:  0.5713376472760041

month:  5
Train Size:  (8004, 3380)
Test Size:  (744, 3380)
Train R2:  0.9055000741944782
Test R2:  0.5729797693219236
Edge R2:  0.5729797693219236

month:  6
Train Size:  (8028, 3380)
Test Size:  (720, 3380)
Train R2:  0.9060628071645319
Test R2:  0.5518261508837256
Edge R2:  0.5518261508837256

month:  7
Train Size:  (8004, 3380)
Test Size:  (744, 3380)
Train R2:  0.9064519471745631
Test R2:  0.5

In [33]:
print(np.mean(CommR2List))
print(np.mean(EdgeR2List))

0.5449130903193496
0.5449130903193496


In [34]:
res_df = pd.concat(residualDf_list, axis = 0)
print(res_df.shape)
res_df.head()

(8748, 260)


,Date,Hour,1,10,100,101,102,106,107,108,...,90,91,92,93,94,95,96,97,98,99
12,2018-01-01,12,-0.217778,-0.307778,-3.355556,0.341111,0.293333,-0.825556,3.883333,-0.497778,...,0.158889,-0.723333,-0.654444,2.502222,-0.096667,0.580000,-0.083333,4.070000,-0.911111,-0.006667
13,2018-01-01,13,-0.574444,0.243333,-1.045556,1.111111,-0.983333,-0.134444,1.901111,1.417778,...,4.341111,-1.188889,1.730000,0.416667,-0.117778,-1.981111,-0.120000,4.102222,-1.515556,-0.005556
14,2018-01-01,14,0.246667,-2.325556,-5.365556,1.931111,-1.085556,-1.304444,8.923333,-0.443333,...,4.944444,1.526667,-1.410000,1.380000,-0.148889,2.321111,0.890000,3.497778,-0.722222,-0.004444
15,2018-01-01,15,1.306667,2.084444,-2.641111,-0.103333,-1.156667,2.561111,-2.141111,0.428889,...,5.970000,-0.682222,-0.027778,-0.604444,-0.180000,3.507778,-0.172222,3.798889,-0.670000,-0.013333
16,2018-01-01,16,0.448889,0.614444,-2.744444,-1.033333,-0.210000,-1.437778,3.762222,-0.552222,...,8.920000,-0.516667,2.277778,0.375556,-0.154444,-0.677778,-0.121111,1.645556,-1.474444,0.000000


In [37]:
dateCols = ['Date', 'Hour']
otherCols = [c for c in res_df.columns if c not in dateCols]

res_df['residual'] = np.sum(res_df[otherCols].values, axis=1)

res_df = res_df[dateCols+['residual']]
print(res_df.shape)

            Date  Hour    residual
12    2018-01-01    12   58.634444
13    2018-01-01    13  -51.757778
14    2018-01-01    14  119.351111
15    2018-01-01    15  105.615556
16    2018-01-01    16   84.226667
17    2018-01-01    17   55.403333
18    2018-01-01    18  -79.283333
19    2018-01-01    19  101.992222
20    2018-01-01    20 -194.812222
21    2018-01-01    21  340.793333
22    2018-01-01    22  372.237778
23    2018-01-01    23  151.171111
24    2018-01-02     0 -199.312222
25    2018-01-02     1 -153.616667
26    2018-01-02     2 -112.723333
27    2018-01-02     3 -106.292222
28    2018-01-02     4   26.444444
29    2018-01-02     5  170.775556
30    2018-01-02     6  285.501111
31    2018-01-02     7    1.186667
32    2018-01-02     8   87.631111
33    2018-01-02     9  -13.704444
34    2018-01-02    10  145.344444
35    2018-01-02    11  -86.628889
36    2018-01-02    12  -44.345556
37    2018-01-02    13   19.070000
38    2018-01-02    14  -12.728889
39    2018-01-02    

In [39]:
res_df.to_csv('/home/urwa/Documents/Projects/NYU Remote/project/data/residuals/jfk_edge.csv')